In [1]:
import os

import numpy as np
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

import networkx as nx
import pickle

In [4]:
n = 2**7
setting = f'n{n}'

G = nx.balanced_tree(2, int(np.log2(n)))

for i in G.nodes():
    neighbors = list(G.neighbors(i))
    G.nodes[i]['children'], G.nodes[i]['parent'] = [x for x in neighbors if x > i], [x for x in neighbors if x < i]

for i in G.nodes():
    G.nodes[i]['obj'] = 'staff'
    G.nodes[i]['param'] = {}

leaves = [x for x in G.nodes() if len(G.nodes[x]['children']) == 0]
for i in list(G.nodes())[::-1]:
    if len(G.nodes[i]['children']) == 0:
        G.nodes[i]['param']['w'] = i - leaves[0] + 1
    else:
        G.nodes[i]['param']['w'] = G.nodes[G.nodes[i]['children'][0]]['param']['w'] + G.nodes[G.nodes[i]['children'][1]]['param']['w']

levels = {0:0}
for i in G.nodes():
    if len(G.nodes[i]['parent']) == 0:
        continue
    levels[i] = levels[G.nodes[i]['parent'][0]] + 1

R = n*100 # 100 members for each task on average
G.nodes[0]['l'] = R
G.nodes[0]['u'] = R

for i in list(G.nodes())[:0:-1]:
    if len(G.nodes[i]['children']) == 0:
        G.nodes[i]['l'] = 1
        G.nodes[i]['u'] = R
    else:
        l1, l2 = G.nodes[G.nodes[i]['children'][0]]['l'], G.nodes[G.nodes[i]['children'][1]]['l']
        G.nodes[i]['l'] = l1 + l2
        G.nodes[i]['u'] = R

ab_list = [(1,10), (1,50), (5,10), (5,50), (10,10), (10,50), (20,10), (20,50)]

# add random weights
for alpha, beta in ab_list:

    new_directory_path = f'data/{setting}/o{alpha}_c{beta}'
    if not os.path.exists(new_directory_path):
        os.makedirs(new_directory_path)

    for t in range(1000):
        Gt = G.copy()
        for i in G.nodes():
            Gt.nodes[i]['param']['w'] += np.random.randn(1)[0] * alpha
            Gt.nodes[i]['param']['w'] = max(Gt.nodes[i]['param']['w'], 1)
            Gt.nodes[i]['l'] += np.random.randint(0, beta+1) 
            Gt.nodes[i]['l'] = min(Gt.nodes[i]['l'], int(R/2**(levels[i]))) # take min to ensure feasibility
        with open(f'{new_directory_path}/{t}.pkl', 'wb') as f:
            pickle.dump(Gt, f)
    
    #save params
    np.savez(f'{new_directory_path}/params.npz', n=n, R=R)